In [1]:
import re
import sys
import json

import urllib3
import configparser

import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

from math import ceil
from tqdm import tqdm
from bson import InvalidBSON
from pymongo import MongoClient
from bson.json_util import dumps
from elasticsearch import Elasticsearch

In [ ]:
print('Downloading pre-trained embeddings from tensorflow hub...')
tf.disable_eager_execution()
embed = hub.Module('../out/models/universal-sentence-encoder')
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print('Done.')

In [ ]:
print('Creating tensorflow session...')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print('Done.')

In [17]:
urllib3.disable_warnings()

config = configparser.ConfigParser()
config.read('../server.ini');

In [ ]:
# Elastic
es = Elasticsearch(
    config['ELASTIC']['path'],
    verify_certs=False
);

In [19]:
# Mongo
client = MongoClient(config['MONGODB']['path'])
db = client['swagger']
swagger = db['apis']

In [20]:

tot = 0
skipped = 0

count = swagger.count_documents({})

for n in tqdm(range(count)):
    # try:
        document = swagger.find({}).skip(n).limit(1)[0]
        nl_tags = r'(?:\'|\")(?:description|name|title|summary)(?:\'|\"):\s(?:\'|\")([^\'\"]+)(?:\'|\")'
        # for document in enumerate(list(documents)):
        # try:
        # doc = documents.next()
        # print(doc_n)
        # document = json.loads(dumps(documents.next()))
        # 
        # obj = {
        #     'id': str(document['_id']['$oid']),
        #     'url': str(document['_API_reference']),
        #     'name': str(document['_name']),
        #     'version': str(document['_version']),
        #     'embedding': session.run(embeddings, feed_dict={text_ph: [' '.join(re.findall(nl_tags, str(document)))]}).tolist()[0]
        # }
        # 
        # es.index(index='migration-test', body=obj)
        # 
        # tot += 1
        # sys.stdout.write("\rDoing thing %i" % tot)
        # sys.stdout.flush()
        # break
    # except InvalidBSON:
    #     print(n)
    #     skipped += 1
    # except TypeError:
    #     skipped += 1

# print(tot, '/', count)
# print(skipped, '/', count)

  0%|          | 930/432265 [00:00<06:31, 1102.05it/s]


InvalidBSON: collection must be an instance of str

In [ ]:
query = 'eating healthy'

knn_query = {
    'knn': {
        'field': 'embedding',
        'query_vector': session.run(embeddings, feed_dict={text_ph: [query]}).tolist()[0],
        'k': 5,
        'num_candidates': 10000
    }
}

results = es.search(index='migration-test', body=knn_query, _source=['name', 'version'])['hits']['hits']

In [ ]:
print(f'These are the top 5 results of the query "{query}":\n')

for ind, result in enumerate(results):
    length_name = max([len(el['_source']['name']) for el in results]) + 2
    length_version = max([len(el['_source']['version']) for el in results]) + 2

    print(
        f'\t{str(ind + 1) + ".": <2} {result["_source"]["name"]: <{length_name}} v.{result["_source"]["version"]: <{length_version}} [{str(result["_score"] * 100)[:2]}%]')